# Homework 8
Currently breaking it up into subsections that I can work on

QUESTIONS:

Should you be able to upload database files also??

In [181]:
from flask import Flask, render_template, request, url_for, redirect, flash
import flask
import pybtex.database as pb
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.secret_key = 'Super Secret PW'
db=SQLAlchemy(app)

#TODO: somehow move at least some of this to another file?
class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ref_tag = db.Column(db.String)
    Collection = db.Column(db.String)
    #Adsnote = db.Column(db.String)
    #Adsurl = db.Column(db.String)
    Author = db.Column(db.String)
    #Date_Added = db.Column(db.Date)
    #Date_Modified = db.Column(db.Date)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    #Month = db.Column(db.String)
    Pages = db.Column(db.String) #I think?
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)
    #Bdsk_File_1  = db.Column(db.String)
    
db.create_all()

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


In [182]:
#test out pybtex
import pybtex.database as pb

    
def add_entry_to_db(key, entry, collection):
    keywords = {}
    for field in ['Title', 'Author','Journal', 'Keywords', 'Pages', 'Volume', 'Year']:
        if field in entry.fields:
            keywords[field] = entry.fields[field]
        else:
            keywords[field] = None
    db.session.add(Citation(**keywords, Collection = collection, ref_tag = key))
    db.session.commit()
x=pb.parse_file('hw_8_data/homework_8_refs.bib')
for k,v in x.entries.items():
    add_entry_to_db(k, v, 'col')

In [187]:
[c.Title for c in Citation.query.filter(Citation.Year < 1980).all()]

['{Reddenings of Cepheids using BVI photometry}',
 '{1777 variables in the Magellanic Clouds}',
 '{Periods of 25 Variable Stars in the Small Magellanic Cloud.}']

In [199]:
res = db.session.execute('SELECT * FROM bibliography WHERE Year < 1980')

In [200]:
for row in res:
    print(row['Title'])

{Reddenings of Cepheids using BVI photometry}
{1777 variables in the Magellanic Clouds}
{Periods of 25 Variable Stars in the Small Magellanic Cloud.}


# Part 1: Basic Flask Website Architecture

In [79]:
%%writefile templates/base.html
<!DOCTYPE html>
<html>
    <head>
        <title>My Super Site</title>
    </head>

    <body>
        <h1>{{ page_title }}</h1>
        
        <h2>
    <a href="{{ url_for('home') }}">Home</a>
    <a href="{{ url_for('query_db') }}">Query Database</a>
    <a href="{{ url_for('upload_file') }}">Upload a File</a>
    </h2>

    {{ message | safe}}        
    {% block content %}{% endblock %}        
    </body>
</html>


Overwriting templates/base.html


In [218]:
%%writefile templates/form.html
{% extends "base.html" %}
{% block content %}
            <form action="/query_database/" method="POST">
            Make a query: 
            <input type="text" name="query" />
            <input type="submit" />
            </form>
            {% with messages = get_flashed_messages() %}
            {% if messages %}
    <ul class=flashes>
    {% for message in messages %}
      <li>{{ message | safe}}</li>
    {% endfor %}
    </ul>
  {% endif %}
{% endwith %}
{% endblock %}

Overwriting templates/form.html


In [34]:
%%writefile templates/file_up.html
{% extends "base.html" %}

{% set page_title = 'Choose file to upload' %}
{% block content %}
   
      <form action = "/upload_file/" method = "POST" 
         enctype = "multipart/form-data">
        <input type = "text" name = "collection_name" />

         <input type = "file" name = "file" />
         <input type = "submit"/>
      </form>
      
{% endblock %}

Overwriting templates/file_up.html


In [219]:
%%writefile basic_site.py

from flask import Flask, render_template, request, url_for, redirect, flash
import flask
import pybtex.database as pb
from flask_sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.debug = True
app.secret_key = 'Super Secret PW'
db=SQLAlchemy(app)

#TODO: somehow move at least some of this to another file?
class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ReferenceTag = db.Column(db.String)
    Collection = db.Column(db.String)
    #Adsnote = db.Column(db.String)
    #Adsurl = db.Column(db.String)
    Author = db.Column(db.String)
    #Date_Added = db.Column(db.Date)
    #Date_Modified = db.Column(db.Date)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    #Month = db.Column(db.String)
    Pages = db.Column(db.String) #I think?
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)
    #Bdsk_File_1  = db.Column(db.String)
    
db.create_all()

@app.route("/")
@app.route("/home")
def home():
    return render_template('base.html', page_title="Home", message="Welcome! </p> this is my site")

@app.route('/query_database/', methods=['GET', 'POST'])
def query_db():
    if request.method == 'POST':
        query = request.form['query']
        #empty form submission doesn't return None, so don't check for that
        if query not in (""," "):
            result=db.engine.execute(query).fetchall()
            if len(result) > 0:
                display_query_results(x)
            else:
                flash('No results to display :(')
            return render_template('form.html', page_title= 'Results for : ' + query)
        else:
            return render_template('form.html', page_title= 'Query Database', 
                                   message='Invalid query. Please try again')
    else:
        ## this is a normal GET request
        return render_template("form.html", page_title = 'Query a Database :)')
    
    
    
@app.route('/upload_file/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template('file_up.html', message = 'Please Choose a File')
        file = request.files['file']
        if file not in (""," "):
            coll_name = request.form['collection_name']
            if coll_name not in (""," ",None):
                #read and convert to this database thing.
                x=pb.parse_bytes(file.read(), bib_format='bibtex')
                for k, entry in x.entries.items():
                    add_entry_to_db(k, entry, coll_name)
                return render_template('base.html', page_title= coll_name)
            else:
                return render_template('file_up.html',
                                       message = 'Please enter a valid collection name')
        else:
            return render_template('file_up.html', message = 'Please Choose a File')
    else:
        ## this is a normal GET request
        return render_template("file_up.html", page_title = 'Upload a File :)')

def display_query_results(results):
    fields = ['ReferenceTag','Collection', 'Title', 'Author','Journal', 'Keywords',
              'Pages', 'Volume', 'Year']
    for row in results:
        for field in fields:
            print(field)
            flash(field + ': ' + str(row[field]))
        flash('</br>')
    
    
def add_entry_to_db(key, entry, collection):
    keywords = {}
    for field in ['Title', 'Author','Journal', 'Keywords', 'Pages', 'Volume', 'Year']:
        if field in entry.fields:
            keywords[field] = entry.fields[field]
        else:
            keywords[field] = None
    db.session.add(Citation(**keywords, Collection = collection, ReferenceTag = key))
    db.session.commit()
    
if __name__ == "__main__":
    app.run()

Overwriting basic_site.py


In [215]:
%run basic_site.py

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '


OSError: [Errno 98] Address already in use

# Part 2: Add travis testing to Basic Flask

In [220]:
%%writefile testing_test.py
import os
import unittest
from basic_site import app

class FlaskrTestCase(unittest.TestCase):

    def setUp(self):
        #self.db_fd, flaskr.app.config['DATABASE'] = tempfile.mkstemp()
        app.testing = True
        self.app = app.test_client()
        #with flaskr.app.app_context():
        #    flaskr.init_db()

    def tearDown(self):
        #os.close(self.db_fd)
        #os.unlink(flaskr.app.config['DATABASE'])
        pass
    
    def test_home_page(self):
        rv = self.app.get('/')
        assert b'Welcome!' in rv.data
    
    def submit_query(self, query):
        #helper function to test querying
        return self.app.post('/query_database/', data=dict(
            query=query
        ), follow_redirects=True)
    
    def test_query_submission(self):
        """tests invalid queries and queries to empty databases"""
        assert b'Results for' in self.submit_query('SELECT * from bibliography').data
        assert b'No results' in self.submit_query('SELECT * from bibliography').data
        assert b'Invalid' in self.submit_query('').data
        assert b'Invalid' in self.submit_query(' ').data        

if __name__ == '__main__':
    unittest.main()

Overwriting testing_test.py


In [221]:
!python testing_test.py

/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:774: UserWarning: Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. Defaulting SQLALCHEMY_DATABASE_URI to "sqlite:///:memory:".
  'Neither SQLALCHEMY_DATABASE_URI nor SQLALCHEMY_BINDS is set. '
/home/hayley/.conda/envs/my_root/lib/python3.6/site-packages/flask_sqlalchemy/__init__.py:794: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  'SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and '
..
----------------------------------------------------------------------
Ran 2 tests in 0.269s

OK


# Part 3: Build a Database

In [125]:
%%writefile database.py

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()
 

class Citation(db.Model):
    __tablename__ = 'bibliography'
    id = db.Column(db.Integer, primary_key=True)
    ref_tag = db.Column(db.String)
    Collection = db.Column(db.String)
    Adsnote = db.Column(db.String)
    Adsurl = db.Column(db.String)
    Author = db.Column(db.String)
    Date_Added = db.Column(db.Date)
    Date_Modified = db.Column(db.Date)
    Journal = db.Column(db.String)
    Keywords = db.Column(db.String)
    Month = db.Column(db.String)
    Pages = db.Column(db.String) #I think?
    Title = db.Column(db.String)
    Volume = db.Column(db.Integer)
    Year = db.Column(db.Integer)
    Bdsk_File_1  = db.Column(db.String)

Overwriting database.py


# Part 4: Add travis to database

# Part 5: Combine database + Flask

# Part 6: add travis to that.